In [1]:


import os
import sys
from datetime import datetime
cur_dir = os.getcwd()
print(f'cur_dir is: {cur_dir}')
pkg_rootdir = os.path.dirname(os.path.dirname(cur_dir))
src_dir = os.path.join(pkg_rootdir, 'src')
if pkg_rootdir not in sys.path:
    sys.path.append(pkg_rootdir)
if src_dir not in sys.path:
    sys.path.append(src_dir)

from core.analysis import analyze_stock_no_parse
from services.ai_service import ai_search
import logging
from services.tushare import find_stock_code
from utils.common import time_now
from services.search import SearchResultItem

log = logging.getLogger(__name__)
log.info('imports loaded')

cur_dir is: /Users/cgl/codes/python/long-quant/src/ipynbs


2025-06-22 23:47:12 INFO [3681102577.py]: imports loaded


In [2]:
def ai_search_workflow(query):
    res_json = ai_search(query, 1, 3)
    time = datetime.now()
    time_str = time.strftime('%m%d_%H%M')
    filename = f"{query[:8]}_{time_str}.md"
    save_path = f"/Users/cgl/Library/Mobile Documents/iCloud~md~obsidian/Documents/md/ask_ai/ai_search/{filename}"
    references_path =  f"/Users/cgl/Library/Mobile Documents/iCloud~md~obsidian/Documents/md/ask_ai/ai_search/references/{filename}"
    references_content = ''
    save_content = f"**问题：** {res_json['query']}\n\n"
    save_content += f"{res_json['answer']}"
    search_result = res_json['search_result']
    log.info(search_result)
    references_content = ''
    for s in search_result:
           references_content += f"### 搜索词：{s['keyword']}\n\n"
           for r in s['results']:
                r: SearchResultItem
                references_content += f"标题: {r.title}\n\n"
                references_content += f"来源链接: {r.url}\n\n"
                references_content += f"内容: {r.content}\n\n"
    with open(save_path, "w") as f:
            f.write(save_content)
    with open(references_path, "w") as f:
            f.write(references_content)
    log.info("AI搜索分析结束，文件已保存")
log.info('ai_search_workflow loaded')

# ai搜索
query="""如何找到细分领域的web或者小程序创业机会"""
ai_search_workflow(query)

2025-06-22 23:47:38 INFO [2373577460.py]: ai_search_workflow loaded
2025-06-22 23:47:39 INFO [_client.py]: HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-22 23:47:39 INFO [ai_service.py]: 搜索关键词： 2025年 细分领域 web小程序 创业机会 发现方法


bocha_search reponse code: 200
response text: {"code":200,"log_id":"dc031809436a61f1","msg":null,"data":{"_type":"SearchResponse","queryContext":{"originalQuery":"2025年 细分领域 web小程序 创业机会 发现方法"},"webPages":{"webSearchUrl":"https://bochaai.com/search?q=2025年 细分领域 web小程序 创业机会 发现方法","totalEstimatedMatches":10000000,"value":[{"id":"https://api.bochaai.com/v1/#WebPages.0","name":"2022-2028年微信小程序行业市场机会挖掘与投资策略咨询报告 - MBA智库文档","url":"https://doc.mbalib.com/view/9e8ddf467895b21b6d54bda93ca41b64.html","displayUrl":"https://doc.mbalib.com/view/9e8ddf467895b21b6d54bda93ca41b64.html","snippet":"2022-2028年微信小程序行业市场机会挖掘与投资策略咨询报告价格RMB 888服务形式PDF+WORD电子版报告简介微信小程序行业都有哪些投资机会?微信小程序行业的细分领域都有哪些投资机会?微信小","summary":"2022-2028年微信小程序行业市场机会挖掘与投资策略咨询报告价格RMB 888服务形式PDF+WORD电子版报告简介微信小程序行业都有哪些投资机会?微信小程序行业的细分领域都有哪些投资机会?微信小程序行业的区域市场都有哪些投资机会?微信小程序行业的产业空白点都有哪些投资机会?微信小程序行业的产业链环节都有哪些投资机会?微信小程序行业投资回报率比较高的投资方向在哪里?当然,最重要的:我们怎么根据这些机会,制定正确的投资策略?本报告将为您一一解答!相信通过本报告对这些问题全面深入的研究和解答,您对这些信息的了解与把控,将上升到一个新的台阶。这将为您经营管理、战略部署、成功投资提供有力的决策参考价

2025-06-22 23:48:07 INFO [_client.py]: HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-22 23:48:07 INFO [2373577460.py]: [{'index': 0, 'keyword': '2025年 细分领域 web小程序 创业机会 发现方法', 'results': [<services.search.SearchResultItem object at 0x1249de660>, <services.search.SearchResultItem object at 0x124a2a990>]}]
2025-06-22 23:48:07 INFO [2373577460.py]: AI搜索分析结束，文件已保存


In [ ]:
def ai_stock_analysis_workflow(ts_code: str=None, name: str=None):
    """
    根据技术面和消息面分析股票，需传入ts_code或者name股票名称至少一个。
    有限根据ts_code
    """
    if ts_code is None:
        if name is None:
            raise ValueError("ts_code和name必须传入一个")
        ts_code = find_stock_code(name)
        if ts_code is None:
            raise ValueError("输入的股票名称不正确")
    prompt, suggestion=analyze_stock_no_parse(ts_code)
    filename = f"stock_{ts_code[:6]}_{time_now('%m%d_%H%M')}.md"
    save_path = f"/Users/cgl/Library/Mobile Documents/iCloud~md~obsidian/Documents/md/ask_ai/stock_analyze/{filename}"
    save_content = f"# 个股分析：{ts_code}"
    save_content += '\n\n'
    save_content += suggestion
    save_content += '\n\n**提问词：**\n'
    save_content += prompt
    with open(save_path, "w") as f:
         f.write(save_content)

# 结合tushare股票最近股价数据和相关新闻快讯，分析股票短中长期投资建议
ai_stock_analysis_workflow(name='电魂网络')

In [ ]:
def loop_analyze():
    for i  in range(5):
        batch_analyze_ranked_stock()